# **Importações**

In [247]:
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps, loads
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt


client = MongoClient('localhost', 27017)
db = client['CovidData']
col = db['Covid']
posts = db.posts

data = pd.read_csv('./owid-covid-data.csv', header=None)


C:\Users\Victo\AppData\Local\Temp\ipykernel_22852\1493425758.py:14: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./owid-covid-data.csv', header=None)


In [91]:
data.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66],
           dtype='int64')

In [92]:
# Pegando a primeira linha para depois inserir como nome das colunas

linha_das_colunas = data.iloc[0]
colunas = []

for item in linha_das_colunas.values:
    colunas.append(item)
    
data.columns = colunas


In [93]:
# Tirando a primeira linha visto que não é mais necessária

data = data.drop(0)

In [94]:
# Vendo o tamanho das colunas pois será necessário para limitar o loop

len(data.columns)

67

In [95]:
# Loop para iterar sobre o csv e transformá-lo em uma lista de dicionários, para assim poder inserir no Mongo

main_fdata = []

for index, row in data.iterrows():
        fdata = {}
        for i in range(67):
                fdata[colunas[i]] = row[colunas[i]]
        main_fdata.append(fdata)

In [196]:
# Inserindo no Mongo

posts.insert_many(main_fdata)

# **Exercício 1 : Liste o nome dos países, seu continente e o total de testes de covid realizados, o total de mortos por covid e o número de vacinados por covid.**

In [217]:
# A agregação dá-se da seguinte forma: Agrupa-se por continent e location, depois é selecionado o valor máximo dos campos especificados com a função $max, mas é preciso fazer uma transformação antes
# pois nos dados da África eles foram colocados como String e não Float, então não é possivel pegar o valor máximo sem transformar para Double


ex1 = db.posts.aggregate([ 
    {"$match": {
    "continent" : {"$ne" : float('NaN')}}},
    {"$group" : {
        "_id": {"continent" : "$continent" , "location" : "$location"}, 
        "totalTests" : {"$max" : {"$toDouble":"$total_tests"}},
        "totalDeaths" : {"$max" : {"$toDouble": "$total_deaths"}},
        "totalpeopleVacc": {"$max": {"$toDouble":"$people_vaccinated"}}}}]
) 

In [218]:
# Transformando o resultado das agregações em uma lista para poder criar um arquivo JSON

ex1_list = list(ex1)

In [219]:
# Criando arquivo JSON

json_data = dumps(ex1_list, indent = 2) 
   
with open('ex1.json', 'w') as file:
    file.write(json_data)

In [220]:
# Lê o JSON como uma lista de dicionários para passar para o dataframe

with open('ex1.json') as json_file:
    teste = json.load(json_file)

In [221]:
# Normaliza o dataframe, já que existe um aninhamento de dicionários, recebe uma lista de dicionários como parâmetro
# Isso é feito com o intuito de separar o aninhamento em colunas diferentes, pois se não for feito isso, a coluna recebe como valor um dicionário

df_ex1_normalizado = pd.json_normalize(teste)

In [222]:
df_ex1_normalizado

,totalTests,totalDeaths,totalpeopleVacc,_id.continent,_id.location,totalTests.$numberDouble,totalDeaths.$numberDouble,totalpeopleVacc.$numberDouble
0,264839.0,138.0,873628.0,Asia,Timor,NaN,NaN,NaN
1,163164533.0,101203.0,57941051.0,Asia,Turkey,NaN,NaN,NaN
2,NaN,6.0,654993.0,Asia,Macao,NaN,NaN,NaN
3,15569464.0,330729.0,99071001.0,North America,Mexico,NaN,NaN,NaN
4,994894.0,7843.0,11320910.0,Asia,Afghanistan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
230,254538.0,314.0,5935299.0,Africa,Niger,NaN,NaN,NaN
231,1618566.0,1928.0,14887847.0,Africa,Angola,NaN,NaN,NaN
232,NaN,13.0,96184.0,Oceania,Kiribati,NaN,NaN,NaN
233,1102099.0,1988.0,1949419.0,Africa,Senegal,NaN,NaN,NaN


In [223]:
# Reorganizando as colunas apenas por questão estética

cols_ex1 = list(df_ex1_normalizado.columns)
cols_ex1.reverse()
df_ex1_normalizado = df_ex1_normalizado[cols_ex1]

In [224]:
# Também renomenando as colunas normalizadas por questão estética

df_ex1_normalizado = df_ex1_normalizado.rename(columns = {"_id.continent" : "continent", "_id.location" : "location"})

In [225]:
# Ordenando o dataframe por continente e local

df_ex1_normalizado = df_ex1_normalizado.sort_values(by=['continent', 'location'])

In [226]:
df_ex1_normalizado

,totalpeopleVacc.$numberDouble,totalDeaths.$numberDouble,totalTests.$numberDouble,location,continent,totalpeopleVacc,totalDeaths,totalTests
98,NaN,NaN,NaN,Algeria,Africa,7840131.0,6881.0,230553.0
231,NaN,NaN,NaN,Angola,Africa,14887847.0,1928.0,1618566.0
165,NaN,NaN,NaN,Benin,Africa,3209393.0,163.0,604310.0
128,NaN,NaN,NaN,Botswana,Africa,1684721.0,2793.0,2285160.0
27,NaN,NaN,NaN,Burkina Faso,Africa,4366726.0,387.0,324773.0
...,...,...,...,...,...,...,...,...
144,NaN,NaN,NaN,Paraguay,South America,3975338.0,19646.0,2609819.0
114,NaN,NaN,NaN,Peru,South America,30154168.0,217738.0,28971116.0
72,NaN,NaN,NaN,Suriname,South America,267820.0,1393.0,84184.0
218,NaN,NaN,NaN,Uruguay,South America,3002718.0,7548.0,6086835.0


In [227]:
# Retirando colunas "lixo"
df_ex1_normalizado.drop(columns=df_ex1_normalizado.columns[0:3], axis=1, inplace=True)
df_ex1_normalizado

,location,continent,totalpeopleVacc,totalDeaths,totalTests
98,Algeria,Africa,7840131.0,6881.0,230553.0
231,Angola,Africa,14887847.0,1928.0,1618566.0
165,Benin,Africa,3209393.0,163.0,604310.0
128,Botswana,Africa,1684721.0,2793.0,2285160.0
27,Burkina Faso,Africa,4366726.0,387.0,324773.0
...,...,...,...,...,...
144,Paraguay,South America,3975338.0,19646.0,2609819.0
114,Peru,South America,30154168.0,217738.0,28971116.0
72,Suriname,South America,267820.0,1393.0,84184.0
218,Uruguay,South America,3002718.0,7548.0,6086835.0


# **Exercício 2 : Liste o nome dos continentes e exiba o total de testes de covid realizados, o total de mortos por covid e o número de vacinados por covid.**

In [147]:
# Após analisar o dataset, foi entendido que quando Continent = NaN e Location = Nome do continente, trata-se de um agrupamento já feito do continente, logo passamos a função $eq NaN para filtramos
# por esse tipo de documento específico



ex2 = db.posts.aggregate([
{"$match": {
    "continent" : float('NaN')
}},
{"$group" : 
        {"_id" : {"location" : "$location"
        },
            "totalDeaths" : {
                "$max": {"$toDouble" : "$total_deaths"}},
            "totalpeopleVacc": {
                "$max" : "$people_vaccinated"
                },
                
        }
    }
     
]) 

In [148]:
# Transformando o resultado das agregações em uma lista para poder criar um arquivo JSON

ex2_list = list(ex2)

In [149]:
# Criando arquivo JSON

json_data2 = dumps(ex2_list, indent = 2) 
   
with open('ex2.json', 'w') as file:
    file.write(json_data2)

In [159]:
# Lê o JSON como uma lista de dicionários para passar para o dataframe

with open('ex2.json') as json_file:
    teste2 = json.load(json_file)

In [160]:
# Normaliza o dataframe, já que existe um aninhamento de dicionários, recebe uma lista de dicionários como parâmetro
# Isso é feito com o intuito de separar o aninhamento em colunas diferentes, pois se não for feito isso, a coluna recebe como valor um dicionário

df_ex2_normalizado = pd.json_normalize(teste2)

In [161]:
df_ex2_normalizado

,totalDeaths,totalpeopleVacc,_id.location,totalpeopleVacc.$numberDouble
0,1544220.0,456723141.0,North America,NaN
1,2760958.0,995644210.0,High income,NaN
2,1989605.0,521676886.0,Europe,NaN
3,1181917.0,337999078.0,European Union,NaN
4,21593.0,28948711.0,Oceania,NaN
5,2507022.0,2100725590.0,Upper middle income,NaN
6,1342856.0,2192560568.0,Lower middle income,NaN
7,1337757.0,373206436.0,South America,NaN
8,47927.0,185171331.0,Low income,NaN
9,15.0,NaN,International,NaN


In [162]:
# Também renomenando as colunas normalizadas por questão estética

df_ex2_normalizado = df_ex2_normalizado.rename(columns = {"_id.location" : "continent"})

In [163]:
# Reorganizando as colunas apenas por questão estética

cols_ex2 = list(df_ex2_normalizado.columns)
cols_ex2.reverse()
df_ex2_normalizado = df_ex2_normalizado[cols_ex2]

In [164]:
df_ex2_normalizado.drop(columns=df_ex2_normalizado.columns[0], axis=1, inplace=True)
df_ex2_normalizado

,continent,totalpeopleVacc,totalDeaths
0,North America,456723141.0,1544220.0
1,High income,995644210.0,2760958.0
2,Europe,521676886.0,1989605.0
3,European Union,337999078.0,1181917.0
4,Oceania,28948711.0,21593.0
5,Upper middle income,2100725590.0,2507022.0
6,Lower middle income,2192560568.0,1342856.0
7,South America,373206436.0,1337757.0
8,Low income,185171331.0,47927.0
9,International,NaN,15.0


In [165]:
# Tirando coluna irrelevantes

df_ex2_normalizado.drop(index = ([1,3,5,6,8,9,12]), axis=0, inplace=True)


In [166]:
df_ex2_normalizado

,continent,totalpeopleVacc,totalDeaths
0,North America,456723141.0,1544220.0
2,Europe,521676886.0,1989605.0
4,Oceania,28948711.0,21593.0
7,South America,373206436.0,1337757.0
10,Africa,9912181.0,257352.0
11,Asia,3634599180.0,1508127.0
